In [31]:
import numpy as np
import cv2 as cv

# Load video
video_path = "volleyball_match.mp4"
cap = cv.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv.CAP_PROP_FPS))

# Set up VideoWriter to save output
output_path = "volleyball_match_output.avi"
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Initialize background subtractor for Team 2
bg_subtractor = cv.createBackgroundSubtractorMOG2()

# HSV Color Ranges
yellow_lower = np.array([10, 100, 100])
yellow_upper = np.array([40, 255, 255])

blue_lower = np.array([100, 150, 50])
blue_upper = np.array([140, 255, 255])

red_lower = np.array([174, 137, 100])
red_upper = np.array([176, 255, 255])

white_lower = np.array([0, 0, 200])  # Adjusted to avoid background
white_upper = np.array([180, 40, 255])

def circularity(area, perimeter):
    if perimeter > 0:
        return (4 * np.pi * area) / (perimeter ** 2)
    return 0

# Processing Loop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    # Creating color masks
    yellow_mask = cv.inRange(hsv, yellow_lower, yellow_upper)
    blue_mask = cv.inRange(hsv, blue_lower, blue_upper)
    red_mask = cv.inRange(hsv, red_lower, red_upper)
    white_mask = cv.inRange(hsv, white_lower, white_upper)

    # Combine yellow and blue for Team 1
    team1_mask = cv.bitwise_or(yellow_mask, blue_mask)

    # Apply background subtraction for Team 2
    fg_mask = bg_subtractor.apply(frame)
    team2_mask = cv.bitwise_or(red_mask, white_mask)
    team2_mask = cv.bitwise_and(team2_mask, team2_mask, mask=fg_mask)

    # Remove noise and small artifacts
    kernel = np.ones((5, 5), np.uint8)
    team1_mask = cv.morphologyEx(team1_mask, cv.MORPH_CLOSE, kernel)
    team1_mask = cv.dilate(team1_mask, kernel, iterations=2)
    team2_mask = cv.morphologyEx(team2_mask, cv.MORPH_CLOSE, kernel)
    team2_mask = cv.dilate(team2_mask, kernel, iterations=2)

    # Find contours for each team
    team1_contours, _ = cv.findContours(team1_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    team2_contours, _ = cv.findContours(team2_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    team1 = 0
    team2 = 0

    def is_valid_player(cnt):
        area = cv.contourArea(cnt)
        x, y, w, h = cv.boundingRect(cnt)
        aspect_ratio = w / float(h)
        return 2000 < area < 5000 and 0.2 < aspect_ratio < 0.9  # Ensures human-like shape

    # Process Team 1 (Yellow + Blue)
    for cnt in team1_contours:
        if is_valid_player(cnt):
            x, y, w, h = cv.boundingRect(cnt)
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)  # Yellow Box
            team1 += 1

    # Process Team 2 (Red + White)
    for cnt in team2_contours:
        if is_valid_player(cnt):
            x, y, w, h = cv.boundingRect(cnt)
            cv.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 255), 2)  # Red Box
            team2 += 1

    # Volleyball Detection
    max_score = 0
    best_contour = None
    for cnt in team1_contours:
        area = cv.contourArea(cnt)
        perimeter = cv.arcLength(cnt, closed=True)
        if 300 < area < 2000:  # Ensure reasonable size
            score = circularity(area, perimeter)
            if 0.7 < score < 1.2:  # Ensures round shape
                if score > max_score:
                    max_score = score
                    best_contour = cnt

    # Draw detected ball
    if best_contour is not None:
        x, y, w, h = cv.boundingRect(best_contour)
        center = (x + w // 2, y + h // 2)
        cv.circle(frame, center, 10, (0, 255, 0), 3)  # Draw red circle

    # Display Player Count
    cv.putText(frame, f"Team 1: {team1}", (50, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv.putText(frame, f"Team 2: {team2}", (50, 100), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)

    # Write frame to output video
    out.write(frame)

    # Show Output
    cv.imshow("Volleyball Player Detection", frame)

    # Exit on 'q' key
    if cv.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv.destroyAllWindows()

print(f"Output saved to {output_path}")


Output saved to volleyball_match_output.avi
